In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7219,2021-02-01T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1336,204,1540,40662,...,NaN,13.0,NaN,NaN,136869.0,0.0,1513602.0,303606.0,ITG,ITG2
7220,2021-02-01T17:00:00,ITA,9,Toscana,43.769231,11.255889,656,101,757,8525,...,NaN,6.0,NaN,NaN,134730.0,99.0,2156970.0,87455.0,ITG,ITG1
7221,2021-02-01T17:00:00,ITA,10,Umbria,43.106758,12.388247,363,58,421,5222,...,Si fa presente come un ricovero (in terapia in...,6.0,NaN,NaN,36194.0,0.0,595877.0,34464.0,ITI,ITI2
7222,2021-02-01T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,33,3,36,193,...,NaN,0.0,NaN,NaN,7800.0,0.0,69280.0,947.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7219,2021-02-01T17:00:00,19,Sicilia,1336,204,1540,40662,42202,-87,766,...,NaN,NaN,136869,1817208,989907.0,13.0,136869.0,0.0,1513602.0,303606.0
7220,2021-02-01T17:00:00,9,Toscana,656,101,757,8525,9282,107,373,...,NaN,NaN,134829,2244425,1199293.0,6.0,134730.0,99.0,2156970.0,87455.0
7221,2021-02-01T17:00:00,10,Umbria,363,58,421,5222,5643,14,143,...,NaN,NaN,36194,630341,283870.0,6.0,36194.0,0.0,595877.0,34464.0
7222,2021-02-01T17:00:00,2,Valle d'Aosta,33,3,36,193,229,-4,3,...,NaN,NaN,7800,70227,41528.0,0.0,7800.0,0.0,69280.0,947.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-02-01', '2021-01-31')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-01T17:00:00,1,Piemonte,2151,153,2304,9960,12264,-48,514,...,NaN,NaN,223611,2083854,1101742.0,12.0,221638.0,1973.0,1886965.0,196889.0
1,2021-02-01T17:00:00,2,Valle d'Aosta,33,3,36,193,229,-4,3,...,NaN,NaN,7800,70227,41528.0,0.0,7800.0,0.0,69280.0,947.0
2,2021-02-01T17:00:00,3,Lombardia,3489,371,3860,44615,48475,-2602,1093,...,NaN,NaN,539147,5682564,2842721.0,26.0,537728.0,1419.0,5545650.0,136914.0
3,2021-02-01T17:00:00,5,Veneto,1410,214,1624,32196,33820,-125,510,...,NaN,NaN,312695,4165203,1324579.0,4.0,311391.0,1304.0,3756472.0,408731.0
4,2021-02-01T17:00:00,6,Friuli Venezia Giulia,593,61,654,10655,11309,-24,129,...,NaN,NaN,67663,1148761,417547.0,10.0,65127.0,2536.0,1112860.0,35901.0
5,2021-02-01T17:00:00,7,Liguria,631,65,696,3870,4566,130,149,...,NaN,NaN,69817,864254,378515.0,5.0,69817.0,0.0,821828.0,42426.0
6,2021-02-01T17:00:00,8,Emilia-Romagna,2181,207,2388,44219,46607,-261,1051,...,NaN,NaN,219702,3113966,1360212.0,8.0,219646.0,56.0,2993949.0,120017.0
7,2021-02-01T17:00:00,9,Toscana,656,101,757,8525,9282,107,373,...,NaN,NaN,134829,2244425,1199293.0,6.0,134730.0,99.0,2156970.0,87455.0
8,2021-02-01T17:00:00,10,Umbria,363,58,421,5222,5643,14,143,...,NaN,NaN,36194,630341,283870.0,6.0,36194.0,0.0,595877.0,34464.0
9,2021-02-01T17:00:00,11,Marche,535,72,607,8565,9172,-86,189,...,NaN,NaN,55676,691642,416358.0,4.0,55676.0,0.0,665623.0,26019.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-31T17:00:00,1,Piemonte,2122,146,2268,10044,12312,-70,538,...,NaN,NaN,223097,2072804,1099174.0,0.0,221177.0,1920.0,1882944.0,189860.0
1,2021-01-31T17:00:00,2,Valle d'Aosta,32,3,35,198,233,-15,16,...,NaN,NaN,7797,70139,41505.0,0.0,7797.0,0.0,69198.0,941.0
2,2021-01-31T17:00:00,3,Lombardia,3503,362,3865,47212,51077,936,1438,...,NaN,NaN,538054,5665413,2837007.0,7.0,536665.0,1389.0,5531042.0,134371.0
3,2021-01-31T17:00:00,5,Veneto,1430,217,1647,32298,33945,-732,499,...,NaN,NaN,312185,4153641,1323060.0,5.0,310868.0,1317.0,3751417.0,402224.0
4,2021-01-31T17:00:00,6,Friuli Venezia Giulia,590,60,650,10683,11333,39,396,...,NaN,NaN,67534,1146864,416815.0,8.0,65047.0,2487.0,1111452.0,35412.0
5,2021-01-31T17:00:00,7,Liguria,612,65,677,3759,4436,41,307,...,NaN,NaN,69668,862130,377791.0,3.0,69668.0,0.0,820447.0,41683.0
6,2021-01-31T17:00:00,8,Emilia-Romagna,2112,206,2318,44550,46868,509,1277,...,NaN,NaN,218651,3103697,1357767.0,10.0,218595.0,56.0,2986157.0,117540.0
7,2021-01-31T17:00:00,9,Toscana,648,96,744,8431,9175,92,510,...,NaN,NaN,134456,2237018,1195439.0,8.0,134360.0,96.0,2151364.0,85654.0
8,2021-01-31T17:00:00,10,Umbria,361,54,415,5214,5629,118,294,...,NaN,NaN,36051,629140,283677.0,2.0,36051.0,0.0,595305.0,33835.0
9,2021-01-31T17:00:00,11,Marche,529,72,601,8657,9258,98,362,...,NaN,NaN,55487,690203,415370.0,4.0,55487.0,0.0,664718.0,25485.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-01T17:00:00,1,Piemonte,2151,153,2304,9960,12264,-48,514,...,NaN,NaN,223611,2083854,1101742.0,12.0,221638.0,1973.0,1886965.0,196889.0
1,2021-02-01T17:00:00,2,Valle d'Aosta,33,3,36,193,229,-4,3,...,NaN,NaN,7800,70227,41528.0,0.0,7800.0,0.0,69280.0,947.0
2,2021-02-01T17:00:00,3,Lombardia,3489,371,3860,44615,48475,-2602,1093,...,NaN,NaN,539147,5682564,2842721.0,26.0,537728.0,1419.0,5545650.0,136914.0
3,2021-02-01T17:00:00,4,Trentino-Alto Adige,413,69,482,6800,7282,195,369,...,0.0,0.0,68063,1030246,343152.0,3.0,61743.0,6320.0,938985.0,91261.0
4,2021-02-01T17:00:00,5,Veneto,1410,214,1624,32196,33820,-125,510,...,NaN,NaN,312695,4165203,1324579.0,4.0,311391.0,1304.0,3756472.0,408731.0
5,2021-02-01T17:00:00,6,Friuli Venezia Giulia,593,61,654,10655,11309,-24,129,...,NaN,NaN,67663,1148761,417547.0,10.0,65127.0,2536.0,1112860.0,35901.0
6,2021-02-01T17:00:00,7,Liguria,631,65,696,3870,4566,130,149,...,NaN,NaN,69817,864254,378515.0,5.0,69817.0,0.0,821828.0,42426.0
7,2021-02-01T17:00:00,8,Emilia-Romagna,2181,207,2388,44219,46607,-261,1051,...,NaN,NaN,219702,3113966,1360212.0,8.0,219646.0,56.0,2993949.0,120017.0
8,2021-02-01T17:00:00,9,Toscana,656,101,757,8525,9282,107,373,...,NaN,NaN,134829,2244425,1199293.0,6.0,134730.0,99.0,2156970.0,87455.0
9,2021-02-01T17:00:00,10,Umbria,363,58,421,5222,5643,14,143,...,NaN,NaN,36194,630341,283870.0,6.0,36194.0,0.0,595877.0,34464.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-01-31T17:00:00,1,Piemonte,2122,146,2268,10044,12312,-70,...,NaN,NaN,223097,2072804,1099174.0,0.0,221177.0,1920.0,1882944.0,189860.0
1,1,2021-01-31T17:00:00,2,Valle d'Aosta,32,3,35,198,233,-15,...,NaN,NaN,7797,70139,41505.0,0.0,7797.0,0.0,69198.0,941.0
2,2,2021-01-31T17:00:00,3,Lombardia,3503,362,3865,47212,51077,936,...,NaN,NaN,538054,5665413,2837007.0,7.0,536665.0,1389.0,5531042.0,134371.0
3,19,2021-01-31T17:00:00,4,Trentino-Alto Adige,401,67,468,6619,7087,-10709,...,0.0,0.0,67694,1026221,341868.0,1.0,61496.0,6198.0,936184.0,90037.0
4,3,2021-01-31T17:00:00,5,Veneto,1430,217,1647,32298,33945,-732,...,NaN,NaN,312185,4153641,1323060.0,5.0,310868.0,1317.0,3751417.0,402224.0
5,4,2021-01-31T17:00:00,6,Friuli Venezia Giulia,590,60,650,10683,11333,39,...,NaN,NaN,67534,1146864,416815.0,8.0,65047.0,2487.0,1111452.0,35412.0
6,5,2021-01-31T17:00:00,7,Liguria,612,65,677,3759,4436,41,...,NaN,NaN,69668,862130,377791.0,3.0,69668.0,0.0,820447.0,41683.0
7,6,2021-01-31T17:00:00,8,Emilia-Romagna,2112,206,2318,44550,46868,509,...,NaN,NaN,218651,3103697,1357767.0,10.0,218595.0,56.0,2986157.0,117540.0
8,7,2021-01-31T17:00:00,9,Toscana,648,96,744,8431,9175,92,...,NaN,NaN,134456,2237018,1195439.0,8.0,134360.0,96.0,2151364.0,85654.0
9,8,2021-01-31T17:00:00,10,Umbria,361,54,415,5214,5629,118,...,NaN,NaN,36051,629140,283677.0,2.0,36051.0,0.0,595305.0,33835.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-02-01T17:00:00,1,Piemonte,2151,153,2304,9960,12264,-48,514,...,NaN,223611,2083854,1101742.0,12.0,221638.0,1973.0,1886965.0,196889.0,4341375
1,2021-02-01T17:00:00,2,Valle d'Aosta,33,3,36,193,229,-4,3,...,NaN,7800,70227,41528.0,0.0,7800.0,0.0,69280.0,947.0,125501
2,2021-02-01T17:00:00,3,Lombardia,3489,371,3860,44615,48475,-2602,1093,...,NaN,539147,5682564,2842721.0,26.0,537728.0,1419.0,5545650.0,136914.0,10103969
3,2021-02-01T17:00:00,4,Trentino-Alto Adige,413,69,482,6800,7282,195,369,...,0.0,68063,1030246,343152.0,3.0,61743.0,6320.0,938985.0,91261.0,1074819
4,2021-02-01T17:00:00,5,Veneto,1410,214,1624,32196,33820,-125,510,...,NaN,312695,4165203,1324579.0,4.0,311391.0,1304.0,3756472.0,408731.0,4907704
5,2021-02-01T17:00:00,6,Friuli Venezia Giulia,593,61,654,10655,11309,-24,129,...,NaN,67663,1148761,417547.0,10.0,65127.0,2536.0,1112860.0,35901.0,1211357
6,2021-02-01T17:00:00,7,Liguria,631,65,696,3870,4566,130,149,...,NaN,69817,864254,378515.0,5.0,69817.0,0.0,821828.0,42426.0,1543127
7,2021-02-01T17:00:00,8,Emilia-Romagna,2181,207,2388,44219,46607,-261,1051,...,NaN,219702,3113966,1360212.0,8.0,219646.0,56.0,2993949.0,120017.0,4467118
8,2021-02-01T17:00:00,9,Toscana,656,101,757,8525,9282,107,373,...,NaN,134829,2244425,1199293.0,6.0,134730.0,99.0,2156970.0,87455.0,3722729
9,2021-02-01T17:00:00,10,Umbria,363,58,421,5222,5643,14,143,...,NaN,36194,630341,283870.0,6.0,36194.0,0.0,595877.0,34464.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-02-01T17:00:00,1,Piemonte,2151,153,2304,9960,12264,-48,514,...,223611,2083854,1101742.0,12.0,221638.0,1973.0,1886965.0,196889.0,4341375,575
1,2021-02-01T17:00:00,2,Valle d'Aosta,33,3,36,193,229,-4,3,...,7800,70227,41528.0,0.0,7800.0,0.0,69280.0,947.0,125501,20
2,2021-02-01T17:00:00,3,Lombardia,3489,371,3860,44615,48475,-2602,1093,...,539147,5682564,2842721.0,26.0,537728.0,1419.0,5545650.0,136914.0,10103969,1036
3,2021-02-01T17:00:00,4,Trentino-Alto Adige,413,69,482,6800,7282,195,369,...,68063,1030246,343152.0,3.0,61743.0,6320.0,938985.0,91261.0,1074819,106
4,2021-02-01T17:00:00,5,Veneto,1410,214,1624,32196,33820,-125,510,...,312695,4165203,1324579.0,4.0,311391.0,1304.0,3756472.0,408731.0,4907704,1016
5,2021-02-01T17:00:00,6,Friuli Venezia Giulia,593,61,654,10655,11309,-24,129,...,67663,1148761,417547.0,10.0,65127.0,2536.0,1112860.0,35901.0,1211357,180
6,2021-02-01T17:00:00,7,Liguria,631,65,696,3870,4566,130,149,...,69817,864254,378515.0,5.0,69817.0,0.0,821828.0,42426.0,1543127,209
7,2021-02-01T17:00:00,8,Emilia-Romagna,2181,207,2388,44219,46607,-261,1051,...,219702,3113966,1360212.0,8.0,219646.0,56.0,2993949.0,120017.0,4467118,634
8,2021-02-01T17:00:00,9,Toscana,656,101,757,8525,9282,107,373,...,134829,2244425,1199293.0,6.0,134730.0,99.0,2156970.0,87455.0,3722729,523
9,2021-02-01T17:00:00,10,Umbria,363,58,421,5222,5643,14,143,...,36194,630341,283870.0,6.0,36194.0,0.0,595877.0,34464.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-02-01T17:00:00,3,Lombardia,3489,371,3860,44615,48475,-2602,1093,...,35.81,-2597,-345,17151,5714.0,52,3643,6.37,0.01082,5.34
1,2021-02-01T17:00:00,8,Emilia-Romagna,2181,207,2388,44219,46607,-261,1051,...,32.65,-331,-226,10269,2445.0,34,1278,10.23,0.02353,4.92
2,2021-02-01T17:00:00,15,Campania,1437,97,1534,60799,62333,-89,994,...,19.21,-107,-407,8417,3456.0,8,1075,11.81,0.01718,3.86
3,2021-02-01T17:00:00,19,Sicilia,1336,204,1540,40662,42202,-87,766,...,34.69,-98,50,32749,4292.0,30,823,2.34,0.01542,2.75
4,2021-02-01T17:00:00,12,Lazio,2373,288,2661,58829,61490,-1711,717,...,34.00,-1700,-226,13099,7481.0,38,2390,5.47,0.01222,3.51
5,2021-02-01T17:00:00,1,Piemonte,2151,153,2304,9960,12264,-48,514,...,26.61,-84,-24,11050,2568.0,11,551,4.65,0.01184,5.15
6,2021-02-01T17:00:00,5,Veneto,1410,214,1624,32196,33820,-125,510,...,21.06,-102,11,11562,1519.0,39,596,4.41,0.01039,6.37
7,2021-02-01T17:00:00,16,Puglia,1411,171,1582,50502,52084,-660,379,...,46.34,-681,-690,4122,2276.0,36,1003,9.19,0.00946,3.08
8,2021-02-01T17:00:00,9,Toscana,656,101,757,8525,9282,107,373,...,19.31,94,-137,7407,3854.0,15,251,5.04,0.01002,3.62
9,2021-02-01T17:00:00,4,Trentino-Alto Adige,413,69,482,6800,7282,195,369,...,65.09,181,-233,4025,1284.0,2,172,9.17,0.03433,6.33


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,69,37,106,65.09
1,Marche,72,71,143,50.35
2,Umbria,58,59,117,49.57
3,Puglia,171,198,369,46.34
4,Lombardia,371,665,1036,35.81
5,Sicilia,204,384,588,34.69
6,Lazio,288,559,847,34.00
7,Friuli Venezia Giulia,61,119,180,33.89
8,Emilia-Romagna,207,427,634,32.65
9,Abruzzo,47,101,148,31.76


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,2021-01-30T17:00:00,22,P.A. Trento,173,35,208,2257,2465,94,230,...,NaN,NaN,27392,526484,161358.0,1.0,25416.0,1976.0,509549.0,16935.0
684,2021-01-31T17:00:00,21,P.A. Bolzano,223,32,255,4363,4618,-10713,467,...,NaN,NaN,40167,497235,180213.0,0.0,36043.0,4124.0,425065.0,72170.0
685,2021-01-31T17:00:00,22,P.A. Trento,178,35,213,2256,2469,4,135,...,NaN,NaN,27527,528986,161655.0,1.0,25453.0,2074.0,511119.0,17867.0
686,2021-02-01T17:00:00,21,P.A. Bolzano,233,34,267,4512,4779,161,316,...,NaN,NaN,40483,500552,181357.0,2.0,36257.0,4226.0,427540.0,73012.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,2021-01-28T17:00:00,4,Trentino-Alto Adige,407,64,471,16232,16703,391,769,...,4.0,11509.0,1022.0,405.0,13.59,2.82,6.68,75.24,-4.48,2.40
340,2021-01-29T17:00:00,4,Trentino-Alto Adige,406,64,470,16763,17233,530,897,...,4.0,11105.0,858.0,531.0,13.62,2.73,8.08,104.55,0.00,3.17
341,2021-01-30T17:00:00,4,Trentino-Alto Adige,389,67,456,17340,17796,563,870,...,9.0,10953.0,1022.0,577.0,14.69,2.56,7.94,85.13,4.69,3.27
342,2021-01-31T17:00:00,4,Trentino-Alto Adige,401,67,468,6619,7087,-10709,602,...,5.0,7324.0,1676.0,-10721.0,14.32,6.60,8.22,35.92,0.00,-60.18


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
